### Z Score Trading

In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
from curl_cffi import requests
session = requests.Session(impersonate='chrome')

In [14]:
def ccy_pair_analysis(tickers=['USDKRW=X', 'EURKRW=X'], years=10, rolling_window=252):
        
    end_date = datetime.now(). strftime('%Y-%m-%d')

    start_date = (datetime.now() - timedelta(days=years*365)).strftime('%Y-%m-%d')

    df = pd.DataFrame()
    
    for ticker in tickers:
    
        temp_df = yf.download(ticker, start=start_date, end=end_date, session=session)
    
        temp_df.columns = [f"{col[0]}_{col[1]}" for col in temp_df.columns]
    
        temp_df = temp_df.reset_index()
        
        temp_df = temp_df.iloc[:, :2]
    
        temp_df[f'{ticker} Daily Returns'] = temp_df[f'Close_{ticker}'].pct_change().fillna(0)

        # Calculating cumulative return of asset
        temp_df[f'{ticker} Cumulative Returns'] = [1.0] + [None] * (len(temp_df) - 1)
        
        for i in range(1, len (temp_df)):
            temp_df[f'{ticker} Cumulative Returns'] = (1 + temp_df[f'{ticker} Daily Returns']).cumprod()
        
        if df.empty:
            df = temp_df
        else:
            df = pd.merge(df, temp_df, on='Date', how='left')

    df[f'Close_{tickers[0]}'] = df[f'Close_{tickers[0]}'].ffill()
    df[f'Close_{tickers[1]}'] = df[f'Close_{tickers[1]}'].ffill()
    
    df[f'{tickers[0]} Daily Returns'] = df[f'{tickers[0]} Daily Returns'].fillna(0)
    df[f'{tickers[1]} Daily Returns'] = df[f'{tickers[1]} Daily Returns'].fillna(0)

    # df[f'{tickers[0]} Cumulative Returns'] = df[f'{tickers[1]} Cumulative Returns'].fillna(0)
    # df[f'{tickers[1]} Cumulative Returns'] = df[f'{tickers[1]} Cumulative Returns'].fillna(0)
    
    # PRICE DIFFERENTIAL CALCULATION
    df['Price Differential'] = df[f'Close_{tickers[1]}'] - df[f'Close_{tickers[0]}']
    price_differential = df['Price Differential'].round(3)

    df['rolling_pd_mean'] = df['Price Differential'].rolling(window=rolling_window).mean()
    df['rolling_pd_std'] = df['Price Differential']. rolling(window=rolling_window).std()
    
    rolling_pd_z_score = (df['Price Differential'] - df['rolling_pd_mean']) / df['rolling_pd_std']
    df['Rolling Z-Score of Price Differential'] = rolling_pd_z_score.round(3)


    # RETURN DIFFERENTIAL CALCULATION
    df['Return Differential'] = df[f'{tickers[1]} Cumulative Returns'] - df[f'{tickers[0]} Cumulative Returns']

    df['rolling_rd_mean'] = df['Return Differential'].rolling(window=rolling_window).mean()
    df['rolling_rd_std'] = df['Return Differential'].rolling(window=rolling_window).std() 

    rolling_rd_z_score = (df['Return Differential'] - df['rolling_rd_mean']) / df['rolling_rd_std']
    df['Rolling Z-Score of Return Differential'] = rolling_rd_z_score.round(3)

    return df


def z_score_returns(df, tickers, z_signal=2):

    df['PD Arb Returns'] = [1.0] + [None] * (len(df) - 1)
    df['R Arb Returns'] = [1.0] + [None] * (len(df) - 1)

    # PD ARB RETURNS
    for i in range (1, len(df)):
        if abs(df.loc[i - 1, 'Rolling Z-Score of Price Differential']) < z_signal or np.isnan(df.loc[i - 1, 'Rolling Z-Score of Price Differential']):
            df.loc[i, 'PD Arb Returns'] = df.loc[i - 1, 'PD Arb Returns']

        elif df.loc[i - 1, 'Rolling Z-Score of Price Differential'] >= z_signal:
            long_return = df.loc[i, f'{tickers[0]} Daily Returns']
            short_return = -1 * df.loc[i, f'{tickers[1]} Daily Returns']
            net_return = long_return + short_return
            new_returns = df. loc[i - 1, 'PD Arb Returns'] * (1 + net_return)
            df. loc[i, 'PD Arb Returns'] = new_returns
        
        elif df.loc[i - 1, 'Rolling Z-Score of Price Differential'] <= -z_signal:
            short_return = -1 * df. loc[i, f'{tickers[0]} Daily Returns']
            long_return = df.loc[i, f'{tickers[1]} Daily Returns']
            net_return = long_return + short_return
            new_returns = df. loc[i - 1, 'PD Arb Returns'] * (1 + net_return)
            df.loc[i, 'PD Arb Returns'] = new_returns

    # R ARB RETURNS
    for i in range (1, len(df)):
        if abs(df.loc[i - 1, 'Rolling Z-Score of Return Differential']) < z_signal or np.isnan(df.loc[i - 1, 'Rolling Z-Score of Return Differential']):
            df.loc[i, 'R Arb Returns'] = df.loc[i - 1, 'R Arb Returns']

        elif df.loc[i - 1, 'Rolling Z-Score of Return Differential'] >= z_signal:
            long_return = df.loc[i, f'{tickers[0]} Daily Returns']
            short_return = -1 * df.loc[i, f'{tickers[1]} Daily Returns']
            net_return = long_return + short_return
            new_returns = df. loc[i - 1, 'R Arb Returns'] * (1 + net_return)
            df. loc[i, 'R Arb Returns'] = new_returns
        
        elif df.loc[i - 1, 'Rolling Z-Score of Return Differential'] <= -z_signal:
            short_return = -1 * df. loc[i, f'{tickers[0]} Daily Returns']
            long_return = df.loc[i, f'{tickers[1]} Daily Returns']
            net_return = long_return + short_return
            new_returns = df. loc[i - 1, 'R Arb Returns'] * (1 + net_return)
            df.loc[i, 'R Arb Returns'] = new_returns

    return df


def make_charts(df, signal, tickers, rolling_window=252):

    z_score_fig = make_subplots(
        rows=3, cols=2, 
        row_heights=[0.33, 0.33, 0.33], 
        vertical_spacing=0.1, subplot_titles=[
            'Closing Prices Over Time',
            f'Rolling {rolling_window} day Z-Score of Price Differential',
            f'Returns Over Time - Price Differential StatArb {signal} STD DEV',
            'Cumulative Returns Over Time',
            f'Rolling {rolling_window} day Z-Score of Returns Differential',
            f'Returns Over Time - Return Differential StatArb {signal} STD DEV'
            ], 
            specs=[
                [{"secondary_y": True}, {}],
                [{}, {"secondary_y": True}], 
                [{}, {}]
            ]
    )

    # Closing Prices Over Time
    z_score_fig.add_trace(go.Scatter(x=df['Date'], y=df[f'Close_{tickers[0]}'], name=tickers[0], line=dict(color='cyan')), row=1, col=1, secondary_y=False)
    z_score_fig.add_trace(go.Scatter (x=df['Date'], y=df[f'Close_{tickers[1]}'], name=tickers[1], line=dict(color='green')), row=1, col=1, secondary_y=True)

    # Rolling Z-Score of Price Differential
    z_score_fig.add_trace(go.Scatter(x=df['Date'], y=df['Rolling Z-Score of Price Differential'], mode='lines', name='Z-score Price Differential'), row=1, col=2)

    # Returns Over Time - Price Differential StatArb
    z_score_fig.add_trace(go.Scatter (x=df['Date'], y=df['PD Arb Returns'], mode='lines', name='PD Arb Returns'), row=2, col=1)
    
    # Cumulative Returns Over Time
    z_score_fig.add_trace(go.Scatter(x=df['Date'], y=df[f'{tickers[0]} Cumulative Returns'], name=tickers[0], line=dict(color='cyan')), row=2, col=2, secondary_y=False) 
    z_score_fig.add_trace(go.Scatter(x=df['Date'], y=df[f'{tickers[1]} Cumulative Returns'], name=tickers[1], line=dict(color='green')), row=2, col=2, secondary_y=True)
    
    # Rolling Z-Score of Returns Differential
    z_score_fig.add_trace(go.Scatter (x=df[ 'Date'], y=df['Rolling Z-Score of Return Differential'], mode='lines', name='Z-score Return Differential'), row=3, col=1)
    
    # Returns Over Time - Return Differential StatArb
    z_score_fig.add_trace(go.Scatter(x=df['Date'], y=df['R Arb Returns'], mode='lines', name='R Arb Returns Over Time'), row=3, col=2)

    # Add Std Dev dashed lines to both Z-Score plots
    std_devs = [1, -1, 2, -2]
    
    for threshold in std_devs:
        z_score_fig.add_hline(y=threshold, line=dict(color='red', dash='dash'), annotation_text=f'{threshold} STD DEV', annotation_position='top left', row=1, col=2)
        z_score_fig.add_hline(y=threshold, line=dict(color='red', dash='dash'), annotation_text=f'{threshold} STD DEV', annotation_position='top left', row=3, col=1)
    
    z_score_fig.update_layout(
        title=dict(text=f'Z-Score of Spot Price/ Return Differentials || STD DEV Signal: {signal} || {rolling_window} Day Rolling Window', x=0.5),
        xaxis=dict(title='Date', tickangle=-45), 
        yaxis=dict(title='Z-Score'), 
        template='plotly_dark', 
        autosize=False, 
        width=1200, 
        height=1200
    )
    
    z_score_fig.update_yaxes(title_text=f'Price {tickers[0]}', row=1, col=1, secondary_y=False)
    z_score_fig.update_yaxes(title_text=f' price {tickers[1]}', row=1, col=1, secondary_y=True)
    z_score_fig.update_yaxes(title_text='Z-Score Spot Difference', row=1, col=2)
    
    z_score_fig.update_yaxes(title_text=f'Cumulative Returns {tickers[0]}',col=2, secondary_y=False)
    z_score_fig. update_yaxes(title_text=f'Cumulative Returns {tickers[1]}', row=2, secondary_y=True)
    
    z_score_fig.show()

    return None

tickers, signal, roll_window = ['GBPUSD=X', 'EURUSD=X'], 1, 252
data = ccy_pair_analysis(tickers=tickers, years=15, rolling_window=roll_window)
data_returns = z_score_returns(data, tickers=tickers, z_signal=signal)
make_charts(df=data_returns, signal=signal, tickers=tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
